In [1]:
import os
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm
from glob import glob

In [2]:
tsv_path = 'transctipt0.tsv'
df = pd.read_csv(tsv_path, sep='\t', header = None)
print(df)
df.columns = ['0', '1', '2']    

                 0      1                     2
0       4aa1fdca33  6a6d1      ००७ मिलको दूरीमा
1       86e521554a  1f680      ००७ मिलको दूरीमा
2       c96b2b1afb  e349f      ००७ मिलको दूरीमा
3       2f7ab41953  294d8  ०११ देखि काङ्ग्रेसको
4       c290e5124b  88901      ०१ सौर द्रव्यमान
...            ...    ...                   ...
157900  17c05d0bdb  06372      ह्रास हुन लाग्दछ
157901  2aa4a03723  79c24      ह्रास हुन लाग्दछ
157902  4cb59eba81  99866      ह्रास हुन लाग्दछ
157903  223df784fc  e7c22           ह्रास हुँदै
157904  fc3b1054b0  95358       ह्वाइटआई भनिन्छ

[157905 rows x 3 columns]


In [16]:
df

,0,1,2,flac_path


In [3]:
audio_root_dir = r'd:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00'

flac_path = glob(os.path.join(audio_root_dir, '**', '*.flac'), recursive=True)

# file_path = (os.path.splitext(os.path.basename(f))[0] for f in flac_path)

print(flac_path)
file_map = {os.path.splitext(os.path.basename(f))[0]: f for f in flac_path}
print(file_map)
# Add full path to dataframe
df['flac_path'] = df['0'].map(file_map)

df['file_id'] = df['flac_path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0] if pd.notnull(x) else None)
# Drop missing entries
df = df[df['flac_path'].notnull()]


['d:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0000c0f43b.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\00014b6780.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0001b0e4b4.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0001cf394a.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\00024e9ee7.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0002f96e3e.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\00033e7a6c.flac', 'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0003991b47.flac', 'd:\\nepali_notetaker_project\\nepali_n

In [81]:
df

,0,1,2,flac_path,file_id
295,0071083d08,97b71,११ वर्षको उमेरमा रक्सी,d:\nepali_notetaker_project\nepali_notetaker_p...,0071083d08
407,0052912514,73d74,१२ वर्षे मकर,d:\nepali_notetaker_project\nepali_notetaker_p...,0052912514
769,00772a60b9,5bc3b,१८३९ देखि १८४२,d:\nepali_notetaker_project\nepali_notetaker_p...,00772a60b9
986,001625cc1d,4cc9e,१९४९ को डिसेम्बरमा,d:\nepali_notetaker_project\nepali_notetaker_p...,001625cc1d
1490,0098d95c74,da6d1,१ हजार अमूल्य,d:\nepali_notetaker_project\nepali_notetaker_p...,0098d95c74
...,...,...,...,...,...
156206,00d83e15f9,c536c,हिसाबले अन्त्यमा ल्याइयो,d:\nepali_notetaker_project\nepali_notetaker_p...,00d83e15f9
156311,00735959ec,f4763,हुत्याउने क्षेत्रभित्र बियो,d:\nepali_notetaker_project\nepali_notetaker_p...,00735959ec
156866,00a154bfae,bb466,हुने वनस्पति हो,d:\nepali_notetaker_project\nepali_notetaker_p...,00a154bfae
157345,001055632c,02e40,हेटौडा हर्नामाडी टाँडी,d:\nepali_notetaker_project\nepali_notetaker_p...,001055632c


In [4]:
from pydub import AudioSegment
from tqdm import tqdm
import os

output_dir = os.path.join(audio_root_dir, 'processed_wav')
os.makedirs(output_dir, exist_ok=True)

wav_paths = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    input_path = row['flac_path']
    print(repr(input_path))
    print(input_path)
    file_id = row['file_id']
    output_path = os.path.join(output_dir, file_id + '.wav')
    print(output_path)
    try:
        # Load and convert to mono 16kHz
        sound = AudioSegment.from_file(input_path, format='flac')
        sound = sound.set_frame_rate(16000).set_channels(1)
        sound.export(output_path, format='wav')
        wav_paths.append(output_path)
    except Exception as e:
        print(f"❌ Failed to process {input_path}: {e}")
        wav_paths.append(None)

df['wav_path'] = wav_paths
df = df[df['wav_path'].notnull()]

# Save only required columns
final_df = df[['wav_path', '2']]
final_df
final_df.columns = ['path', '2']
final_df
final_df.to_csv('final_nepali_dataset.csv', index=False)

print("✅ Conversion complete. CSV saved as final_nepali_dataset.csv.")


  0%|          | 0/607 [00:00<?, ?it/s]

'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0071083d08.flac'
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\0071083d08.flac
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\processed_wav\0071083d08.wav


  0%|          | 2/607 [00:03<14:37,  1.45s/it]

'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0052912514.flac'
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\0052912514.flac
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\processed_wav\0052912514.wav


  0%|          | 3/607 [00:03<08:43,  1.15it/s]

...

'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\001055632c.flac'
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\001055632c.flac
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\processed_wav\001055632c.wav


100%|██████████| 607/607 [02:05<00:00,  5.11it/s]

'd:\\nepali_notetaker_project\\nepali_notetaker_package\\DataSet\\asr_nepali_0\\asr_nepali\\data\\00\\0099d2bf19.flac'
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\0099d2bf19.flac
d:\nepali_notetaker_project\nepali_notetaker_package\DataSet\asr_nepali_0\asr_nepali\data\00\processed_wav\0099d2bf19.wav


100%|██████████| 607/607 [02:05<00:00,  4.82it/s]

✅ Conversion complete. CSV saved as final_nepali_dataset.csv.


In [31]:
df

,0,1,2,flac_path,wav_path
